<a href="https://colab.research.google.com/github/lrobsky/HW2/blob/master/HW1_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# @title imports
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import numpy as np
from collections import namedtuple
from torchvision.utils import make_grid

In [45]:
# @title load cuda
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [46]:
# @title utility functions
# Sample image in dataset
def view_data_sample(loader):
  image, label = next(iter(loader))
  plt.figure(figsize=(16,8))
  plt.axis('off')
  plt.imshow(make_grid(image, nrow=16).permute((1, 2, 0)))


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def splice_batch(X, Y, num_of_labels, prints=False):
  if prints:
    print('input: ',end="")
    print("\t X shape: ", X.shape, end='\t')
    print("\t Y shape: ", Y.shape)
  X = X[Y<num_of_labels]
  Y = Y[Y<num_of_labels]
  if prints:
    print('output: ',end="")
    print("\t X shape: ", X.shape, end='\t')
    print("\t Y shape: ", Y.shape)
  return X, Y

In [47]:
# @title data loading
batch_size = 100
# Download and load the training data
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=ToTensor())
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,pin_memory=True, num_workers=2)
train_loader = [(images.to(device), labels.to(device)) for images, labels in train_loader] # enable CUDA
# Download and load the test data
testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True,pin_memory=True, num_workers=2)
test_loader = [(images.to(device), labels.to(device)) for images, labels in test_loader]  # enable CUDA

In [48]:
# @title Net1
class Net_1(nn.Module):
    def __init__(self):
        super(Net_1, self).__init__()
        self.l1 = nn.Linear(in_features=28*28, out_features=63)  # Mnist = 28x28
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(in_features=63, out_features=3)  # Output size is 3 for 3 classes

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [49]:
# @title Net1 Train&Test
# Net_1:

# spliced_data, spliced_labels = splice_batch(trainset.data, trainset.targets, 3, True) # only 3 classes

model = Net_1()
model = model.to(device)

print(f"{count_parameters(model)}")
weights_path = 'net_1_weights.pth'
# model.load_state_dict(torch.load(weights_path)) # Load the weights

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 30
num_classes = 3
training_accuracy = []
test_accuracy = []
model.eval()

for epoch in range(num_epochs):
    # calculating the network
    for i, (images, labels) in enumerate(train_loader):
        images,labels = images.to(device),labels.to(device)
        # 100, 1, 28, 28
        # 100, 724
        (im,lab) = splice_batch(images,labels,num_classes)
        im = im.reshape(-1,28*28)

        # forward
        outputs = model(im)
        loss = criterion(outputs, lab)

        # backwards
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # if(i%100) == 0:
        #     print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} loss = {loss.item():.4f}")
    with torch.no_grad():
        training_n_correct = 0
        training_n_samples = 0
        for images, labels in train_loader:

            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            training_n_samples += lab.shape[0]
            training_n_correct += (predictions == lab).sum().item()

        training_acc = 100 * training_n_correct / training_n_samples
        training_accuracy.append(training_acc)
        print(f"Training: Epoch #{epoch}: accuracy = {training_acc}")

        test_n_correct = 0
        test_n_samples = 0
        for images, labels in test_loader:

            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            test_n_samples += lab.shape[0]
            test_n_correct += (predictions == lab).sum().item()

        test_acc = 100 * test_n_correct / test_n_samples
        test_accuracy.append(test_acc)
        print(f"Test: Epoch #{epoch}: accuracy = {test_acc}")

with open(r'accuracies1.txt', 'w') as fp:
    fp.write("Training Accuracy\n")
    for acc in training_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    fp.write("\n Test accuracy\n")
    for acc in test_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    print('Done')



49647
Training: Epoch #0: accuracy = 97.29444444444445
Test: Epoch #0: accuracy = 96.4
Training: Epoch #1: accuracy = 97.48333333333333
Test: Epoch #1: accuracy = 96.86666666666666
Training: Epoch #2: accuracy = 97.39444444444445
Test: Epoch #2: accuracy = 96.53333333333333
Training: Epoch #3: accuracy = 97.35555555555555
Test: Epoch #3: accuracy = 96.26666666666667
Training: Epoch #4: accuracy = 97.66666666666667
Test: Epoch #4: accuracy = 96.83333333333333
Training: Epoch #5: accuracy = 97.17222222222222
Test: Epoch #5: accuracy = 96.43333333333334
Training: Epoch #6: accuracy = 97.12777777777778
Test: Epoch #6: accuracy = 96.03333333333333
Training: Epoch #7: accuracy = 97.83333333333333
Test: Epoch #7: accuracy = 97.1
Training: Epoch #8: accuracy = 97.75555555555556
Test: Epoch #8: accuracy = 96.8
Training: Epoch #9: accuracy = 97.73333333333333
Test: Epoch #9: accuracy = 96.8
Training: Epoch #10: accuracy = 97.64444444444445
Test: Epoch #10: accuracy = 96.83333333333333
Training: 

In [50]:
# @title Net2
class Net_2(nn.Module):
    def __init__(self):
        super(Net_2, self).__init__()
        self.l1 = nn.Linear(in_features=28*28, out_features=63)  # Mnist = 28x28
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(in_features=63, out_features=7)  # Output size is 7 for 7 classes

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [ ]:
# @title Net2 Train&Test
# Net_2:

# spliced_data, spliced_labels = splice_batch(trainset.data, trainset.targets, 3, True) # only 3 classes

model = Net_2()
model = model.to(device)
print(f"{count_parameters(model)}")
weights_path = 'net_2_weights.pth'
# model.load_state_dict(torch.load(weights_path)) # Load the weights

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 20
num_classes = 7
training_accuracy = []
test_accuracy = []
model.eval()
for epoch in range(num_epochs):
    # calculating the network
    for i, (images, labels) in enumerate(train_loader):
        # 100, 1, 28, 28
        # 100, 724
        (im,lab) = splice_batch(images,labels,num_classes)
        im = im.reshape(-1,28*28)

        # forward
        outputs = model(im)
        loss = criterion(outputs, lab)

        # backwards
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # if(i%100) == 0:
        #     print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} loss = {loss.item():.4f}")
    with torch.no_grad():
        training_n_correct = 0
        training_n_samples = 0
        for images, labels in train_loader:
            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            training_n_samples += lab.shape[0]
            training_n_correct += (predictions == lab).sum().item()

        training_acc = 100 * training_n_correct / training_n_samples
        training_accuracy.append(training_acc)
        print(f"Training: Epoch #{epoch}: accuracy = {training_acc}")

        test_n_correct = 0
        test_n_samples = 0
        for images, labels in test_loader:
            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            test_n_samples += lab.shape[0]
            test_n_correct += (predictions == lab).sum().item()

        test_acc = 100 * test_n_correct / test_n_samples
        test_accuracy.append(test_acc)
        print(f"Test: Epoch #{epoch}: accuracy = {test_acc}")

with open(r'accuracies2.txt', 'w') as fp:
    fp.write("Training Accuracy\n")
    for acc in training_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    fp.write("\n Test accuracy\n")
    for acc in test_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    print('Done')




In [54]:
# @title Net3
class Net_3(nn.Module):
    def __init__(self):
        super(Net_3, self).__init__()
        self.l1 = nn.Linear(in_features=28*28, out_features=55)  # Mnist = 28x28
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(in_features=55, out_features=56)
        self.l3 = nn.Linear(in_features=56, out_features=56)
        self.l4 = nn.Linear(in_features=56, out_features=7)  # Output size is 7 for 7 classes

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out

In [57]:
# @title Net3 Train&Test
# Net_3:

# spliced_data, spliced_labels = splice_batch(trainset.data, trainset.targets, 3, True) # only 3 classes

model = Net_3()
model = model.to(device)
print(f"{count_parameters(model)}")
weights_path = 'net_3_weights.pth'
# model.load_state_dict(torch.load(weights_path)) # Load the weights

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 30
num_classes = 7
training_accuracy = []
test_accuracy = []
model.eval()
for epoch in range(num_epochs):
    # calculating the network
    for i, (images, labels) in enumerate(train_loader):
        # 100, 1, 28, 28
        # 100, 724
        (im,lab) = splice_batch(images,labels,num_classes)

        im = im.reshape(-1,28*28)

        # forward
        outputs = model(im)
        loss = criterion(outputs, lab)

        # backwards
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # if(i%100) == 0:
        #     print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} loss = {loss.item():.4f}")
    with torch.no_grad():
        training_n_correct = 0
        training_n_samples = 0
        for images, labels in train_loader:
            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            training_n_samples += lab.shape[0]
            training_n_correct += (predictions == lab).sum().item()

        training_acc = 100 * training_n_correct / training_n_samples
        training_accuracy.append(training_acc)
        print(f"Training: Epoch #{epoch}: accuracy = {training_acc}")

        test_n_correct = 0
        test_n_samples = 0
        for images, labels in test_loader:
            (im,lab) = splice_batch(images,labels,num_classes)
            im = im.reshape(-1,28*28)
            outputs = model(im)
            # value, index
            _, predictions = torch.max(outputs,1)
            test_n_samples += lab.shape[0]
            test_n_correct += (predictions == lab).sum().item()

        test_acc = 100 * test_n_correct / test_n_samples
        test_accuracy.append(test_acc)
        print(f"Test: Epoch #{epoch}: accuracy = {test_acc}")

with open(r'accuracies3.txt', 'w') as fp:
    fp.write("Training Accuracy\n")
    for acc in training_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    fp.write("\n Test accuracy\n")
    for acc in test_accuracy:
        # write each item on a new line
        fp.write("%s\n" % acc)
    print('Done')



49902
Training: Epoch #0: accuracy = 81.88095238095238
Test: Epoch #0: accuracy = 80.48571428571428
Training: Epoch #1: accuracy = 83.79285714285714
Test: Epoch #1: accuracy = 82.21428571428571
Training: Epoch #2: accuracy = 84.03809523809524
Test: Epoch #2: accuracy = 82.02857142857142
Training: Epoch #3: accuracy = 83.55238095238096
Test: Epoch #3: accuracy = 81.31428571428572
Training: Epoch #4: accuracy = 84.88571428571429
Test: Epoch #4: accuracy = 82.78571428571429
Training: Epoch #5: accuracy = 84.41904761904762
Test: Epoch #5: accuracy = 82.5
Training: Epoch #6: accuracy = 86.15
Test: Epoch #6: accuracy = 83.68571428571428
Training: Epoch #7: accuracy = 85.34285714285714
Test: Epoch #7: accuracy = 82.82857142857142
Training: Epoch #8: accuracy = 85.70714285714286
Test: Epoch #8: accuracy = 83.4
Training: Epoch #9: accuracy = 86.39285714285714
Test: Epoch #9: accuracy = 83.81428571428572
Training: Epoch #10: accuracy = 85.97857142857143
Test: Epoch #10: accuracy = 83.2
Training: